In [1]:
from py2neo import Graph

url = 'localhost:7474'
username = 'neo4j'
password = 'neo4j'
graph = Graph(url, username=username, password=password)

In [13]:
def networkSearch(value):
    return {
        'colleague': graph.run("match (s:`person`)-[p:`COLLEAGUE_OF`]-(o:`person`) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data(),
        'acknowledge':graph.run("match (s:`person`)-[p:`ACKNOWLEDGE`]-(o:`person`) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data(),
        'parent':graph.run("match (s:`person`)-[p:`PARENT_OF`]-(o:`person`) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data(),
    }
def userInfo(value):
    try:
        rt = graph.run("match (s:`person`) where s.personID= '{}' return s.flag,s.status".format(value)).data()
        return (rt[0]['s.flag'],rt[0]['s.status'])
    except:
        return -1

def network1FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())

def network2FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(:`person`)-[q]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and type(q) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())

def userPhoneNum(value):
    return len(graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where s.personID= '{}' return o.phone".format(value)).data())

def userRmtNum(value):
    # 交易金额大于4000才计数
#     try: 
#         cache = graph.run("match (s:`person`)-[p:`remittance`]-(o:`person`) where s.personID= '{}' return p.amount".format(value)).data()
#         return len([i['p.amount'] for i in cache if int(i['p.amount'])>=4000])
#     except:
#         return -1
    return len(graph.run("match (s:`person`)-[p:`remittance`]-(o:`person`) where s.personID= '{}' and toInt(p.amount)>=4000 return p.amount".format(value)).data())

def userTxConditionNum(value):
    defaultNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='OVER_DUE' and s.personID= '{}' return o.status".format(value)).data())
    clearNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='CLEAR' and s.personID= '{}' return o.status".format(value)).data())
    rejectedNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='REJECT' and s.personID= '{}' return o.status".format(value)).data())
    return defaultNum,clearNum,rejectedNum

In [14]:
# neo4j统计得到的额外数据
def statExData(value):
    try:
        i = userInfo(value=value)
        c1,c2,c3 = userTxConditionNum(value=value)
        return {
        '是否在黑名单':i[0],
        '历史用户评分':i[1],
        '一层关系黑户数':network1FlagNum(value=value),
        '二层关系黑户数':network2FlagNum(value=value),
        '持有手机数':userPhoneNum(value=value),
        '大额转账数':userRmtNum(value=value),
        '违约贷款数':c1,
        '完结贷款数':c2,
        '被拒贷款数':c3,
        }
    except:
        return -1
statExData(value='person_1')

{'是否在黑名单': 0,
 '历史用户评分': 'AA',
 '一层关系黑户数': 0,
 '二层关系黑户数': 0,
 '持有手机数': 1,
 '大额转账数': 7,
 '违约贷款数': 1,
 '完结贷款数': 1,
 '被拒贷款数': 0}

In [4]:
value = 'person_1'
def userRel(value):
    userEntity = graph.run("match (s:`person`) where s.personID= '{}' return s.name, s.personID, s.status, s.flag".format(value)).data()
    if len(userEntity) >0:
        rt = {}
        rt['rels'] = networkSearch(value)
        rt['user'] = userEntity
        relTypes = list(rt['rels'].keys())
        for relType in relTypes:
            for O in rt['rels'][relType]:
                O['rels'] = networkSearch(value=O['o.personID'])
        return rt
    else: return -1
userRel(value)

{'rels': {'colleague': [{'o.name': 'name_7',
    'o.personID': 'person_7',
    'o.status': 'A',
    'o.flag': '0',
    'rels': {'colleague': [{'o.name': 'name_1',
       'o.personID': 'person_1',
       'o.status': 'AA',
       'o.flag': 0}],
     'acknowledge': [{'o.name': 'name_11',
       'o.personID': 'person_11',
       'o.status': 'AA',
       'o.flag': '0'}],
     'parent': [{'o.name': 'name_8',
       'o.personID': 'person_8',
       'o.status': 'B',
       'o.flag': '0'}]}},
   {'o.name': 'name_2',
    'o.personID': 'person_2',
    'o.status': 'A',
    'o.flag': '0',
    'rels': {'colleague': [{'o.name': 'name_1',
       'o.personID': 'person_1',
       'o.status': 'AA',
       'o.flag': 0}],
     'acknowledge': [{'o.name': 'name_3',
       'o.personID': 'person_3',
       'o.status': 'A',
       'o.flag': '0'}],
     'parent': []}}],
  'acknowledge': [{'o.name': 'name_20',
    'o.personID': 'person_20',
    'o.status': 'A',
    'o.flag': '0',
    'rels': {'colleague': [{'o.na

In [7]:
def userPhone(value):
    userEntity = graph.run("match (s:`person`) where s.personID= '{}' return s.name, s.personID, s.status, s.flag".format(value)).data()
    if len(userEntity) >0:
        rt = {}
        rt['user'] = userEntity
        rt['ownPhone'] = graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where s.personID= '{}' return o.number,o.phone,o.flag".format(value)).data()
        return rt
    else: return -1
userPhone(value)

{'user': [{'s.name': 'name_1',
   's.personID': 'person_1',
   's.status': 'AA',
   's.flag': 0}],
 'ownPhone': [{'o.number': 'num_2', 'o.phone': 'phone_2', 'o.flag': '0'}]}

In [8]:
def userTx(value):
    userEntity = graph.run("match (s:`person`) where s.personID= '{}' return s.name, s.personID, s.status, s.flag".format(value)).data()
    if len(userEntity) >0:
        rt = {}
        rt['user'] = userEntity
        rt['apply'] = graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where s.personID= '{}' return o.txID,o.status".format(value)).data()
        return rt
    else: return -1
userTx(value)

{'user': [{'s.name': 'name_1',
   's.personID': 'person_1',
   's.status': 'AA',
   's.flag': 0}],
 'apply': [{'o.txID': 'tx_2', 'o.status': 'OVER_DUE'},
  {'o.txID': 'tx_1', 'o.status': 'CLEAR'}]}

In [20]:
value = 'person_1'
userEntity = graph.run("match (s) where s.personID= '{}' return s.name, s.personID, s.status, s.flag".format(value)).data()
colleagueRels = graph.run("match (s)-[p:`COLLEAGUE_OF`]-(o) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data()
friendRels = graph.run("match (s)-[p:`FRIEND_OF`]-(o) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data()
parentRels = graph.run("match (s)-[p:`PARENT_OF`]-(o) where s.personID= '{}' return o.name,o.personID, o.status, o.flag".format(value)).data()

print(userEntity)
print(colleagueRels)
print(friendRels)
print(parentRels)

[{'s.name': 'name_1', 's.personID': 'person_1', 's.status': 'AA', 's.flag': '0'}]
[{'o.name': 'name_7', 'o.personID': 'person_7', 'o.status': 'A', 'o.flag': '0'}, {'o.name': 'name_2', 'o.personID': 'person_2', 'o.status': 'A', 'o.flag': '0'}]
[{'o.name': 'name_20', 'o.personID': 'person_20', 'o.status': 'A', 'o.flag': '0'}, {'o.name': 'name_10', 'o.personID': 'person_10', 'o.status': 'A', 'o.flag': '0'}]
[{'o.name': 'name_4', 'o.personID': 'person_4', 'o.status': 'AA', 'o.flag': '0'}]


In [154]:
# remittanceRels:
value = 'person_1'

userEntity = graph.run("match (s:`person`) where s.personID= '{}' return s.name, s.personID, s.status, s.flag".format(value)).data()

def remittanceAly(value):
    remittanceRels = graph.run("match (s:`person`)-[p:`remittance`]-(o:`person`) where s.personID= '{}' return p.amount, o.personID".format(value)).data()
    if len(remittanceRels)>0:
        ackUsers = {}
        for rmt in remittanceRels:
            r = 1 if int(rmt['p.amount'])<=10000 else 2
            if rmt['o.personID'] not in ackUsers:
                ackUsers[rmt['o.personID']] = r
            else:
                ackUsers[rmt['o.personID']] += r

        ackUsers = [user for user in ackUsers if ackUsers[user]>1]
        # 最后需要建立关系的id!
        ackUsers = [user for user in ackUsers if 
            # 避免重复建立关系
            len(
                graph.run("match (s:`person`)-[p]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and s.personID= '{}' and o.personID='{}' return type(p)"
                          .format(value,user)).data())
                    ==0]
        if len(ackUsers)>0: return ackUsers
        else:return -1
    else: return -1

remittanceAly(value='person_1')

-1

In [151]:
value = 'person_1'
def callAly(value):
    '''
       流程：
       先得到该personID所持有的所有phone的ID
       查询与该phone有联系的所有phone的ID
       对满足通话次数阈值的phone查询其所有者(user)的ID
       对满足关系但是未建立关系的两个user进行记录并返回
    '''
    ownPhone = [i['o.phone'] for i in 
                graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where s.personID= '{}' return o.phone".format(value))
                .data()]
    if len(ownPhone)>0:
        for phoneID in ownPhone:
            # get phone rel
            callRels = graph.run("match (s:`phone`)-[p:`call`]-(o:`phone`) where s.phone= '{}' return o.phone".format(phoneID)).data()
            ackPhone = {}
            if len(callRels)>0:
                for call in callRels:
                    if call['o.phone'] not in ackPhone:
                        ackPhone[call['o.phone']] = 1
                    else: ackPhone[call['o.phone']] += 1
                # 根据通话次数进行筛选
                ackPhone = [phone for phone in ackPhone if ackPhone[phone]>1]
            
                # ackPhone = ['phone_38', 'phone_35', 'phone_14', 'phone_9', 'phone_5'] # for test
                # phone -> owner
                if len(ackPhone)>0:
                    ackUsers = []
                    for phoneID in ackPhone:
                    
                        for i in graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where o.phone= '{}' return s.personID".format(phoneID)).data():
                            # 避免重复建立关系
                            if len(
                                graph.run("match (s:`person`)-[p]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and s.personID= '{}' and o.personID='{}' return type(p)"
                                  .format(value,i['s.personID'])).data()
                                )==0:
                                ackUsers.append(
                                    i['s.personID']
                                )
                    return ackUsers
                else: return -1        
            else: return -1
    else: return -1
callAly(value)

['person_3']

In [163]:
def getAllUsers():
    users = graph.run("match (s:`person`)  return s.personID").data()
    return [i['s.personID'] for i in users]

In [188]:
userlist = ['person_1','person_10']
def digRel(userlist):
    for userID in userlist:
        userEntity = graph.run("match (s:`person`) where s.personID= '{}' return s.personID".format(value)).data()
        if len(userEntity)>0:
            unrels = []
            rmtRels = remittanceAly(value = value)
            callRels = callAly(value = value)
    
            if  rmtRels != -1:
                unrels += rmtRels
            if callRels != -1:
                unrels += callRels
            unrels = set(unrels)
            for userID in unrels:
                graph.run("match (s:`person`{personID:'" + value +"'}),(o:`person`{personID:'" + userID +"'}) merge (s)-[:ACKNOWLEDGE]->(o)")
            return 1
        else: return -1
digRel(userlist)

1

In [180]:
value = 'person_1'
userID = 'person_10'

In [186]:

"match (s:`person`{personID='" + value +"'}),(o:`person`{personID='" + userID +"'}) merge (s)-[:ACKNOWLEDGE]->(o)"



"match (s:`person`{personID='person_1'}),(o:`person`{personID='person_10'}) merge (s)-[:ACKNOWLEDGE]->(o)"

In [233]:
value1 = 'person_1'
value2 = 'person_6'

def createRmt(value1,value2,amount):
    if str(type(amount)) == "<class 'int'>"  and amount > 0:
        if len(graph.run("match(a:person{personID:'"+value1+"'}),(b:person{personID:'"+value2+"'})return a,b").data())>0:
            graph.run("match (s:`person`{personID:'" + value1 +"'}),(o:`person`{personID:'" + value2 +"'}) merge (s)-[:remittance{amount:'"+ str(amount)+"'}]->(o) return 1")
            return 1
        else:return -1
    else:return -1
    
def createCall(value1,value2):
    if len(graph.run("match(a:person{personID:'"+value1+"'}),(b:person{personID:'"+value2+"'})return a,b").data())>0:
        graph.run("match (s:`person`{personID:'" + value1 +"'}),(o:`person`{personID:'" + value2 +"'}) merge (s)-[:call]->(o) return 1")
        return 1
    else:return -1    
createCall(value1,value2)

1

In [252]:
data = {'address':'广州','age':27,'flag':0,'name':'name_1','personID':'kane'}
cql = ''
for e in data:
    cache = " set a."+e+"='"+str(data[e])+"'"
    cql += cache
cql = "match(a:person{personID:'"+data['personID']+"'})" + cache
cql = "create(a:person)" + cache
graph.run(cql)

In [267]:
def personCU(data, mode='UPDATE'):
    try:
        # CQL: SET
        cache = ''
        for e in data:
            cache += " set a."+e+"='"+str(data[e])+"'"
        if mode=='UPDATE':
            if len(graph.run("match(a:person{personID:'"+data['personID']+"'}) return a").data())>0:
                graph.run("match(a:person{personID:'"+data['personID']+"'})" + cache)
                return 1
            return -1
        elif mode=='CREATE':
            # 不存在才创建
            if len(graph.run("match(a:person{personID:'"+data['personID']+"'}) return a").data())==0:
                graph.run("create(a:person)" + cache)
                return 1
            return -1
        else: return -1
    except: return -1
    
data = {'address':'广州','age':27,'flag':0,'name':'name_1','personID':'jeanne'}
personCU(data=data,mode='CREATE')

1

In [235]:
a = [0]
try:
    if a[0] == 0:
        print(1)
except:
    print(-1)

1


In [268]:
def txCU(data, mode='UPDATE'):
    try:
        # CQL: SET
        cache = ''
        for e in data:
            cache += " set a."+e+"='"+str(data[e])+"'"
        if mode=='UPDATE':
            if len(graph.run("match(a:transaction{txID:'"+data['txID']+"'}) return a").data())>0:
                graph.run("match(a:transaction{txID:'"+data['txID']+"'})" + cache)
                return 1
            return -1
        elif mode=='CREATE':
            # 不存在才创建
            if len(graph.run("match(a:transaction{txID:'"+data['txID']+"'}) return a").data())==0:
                graph.run("create(a:transaction)" + cache)
                return 1
            return -1
        else: return -1
    except: return -1
data = {'status':'CLEAR','txID':'tx_1t'}
data

{'status': 'CLEAR', 'txID': 'tx_1t'}

In [269]:
def phoneCU(data, mode='UPDATE'):
    try:
        # CQL: SET
        cache = ''
        for e in data:
            cache += " set a."+e+"='"+str(data[e])+"'"
        if mode=='UPDATE':
            if len(graph.run("match(a:phone{phone:'"+data['phone']+"'}) return a").data())>0:
                graph.run("match(a:phone{phone:'"+data['phone']+"'})" + cache)
                return 1
            return -1
        elif mode=='CREATE':
            # 不存在才创建
            if len(graph.run("match(a:phone{phone:'"+data['phone']+"'}) return a").data())==0:
                graph.run("create(a:phone)" + cache)
                return 1
            return -1
        else: return -1
    except: return -1
data = {'flag':0,'number':'7777','phone':'phone_1t'}
data

{'flag': 0, 'number': '7777', 'phone': 'phone_1t'}

# 生成数据前置func

In [111]:
value = 'person_1'
userEntity = graph.run("match (s:`person`)  return s.personID,s.flag,s.status").data()
len(userEntity)

200005

In [102]:
def userInfo(value):
    return graph.run("match (s:`person`) where s.personID= '{}' return s.flag,s.status".format(value)).data()

def network1FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())

def network2FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(:`person`)-[q]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and type(q) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())

def userPhoneNum(value):
    return len(graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where s.personID= '{}' return o.phone".format(value)).data())

def userRmtNum(value):
    # 交易金额大于4000才计数
    cache = graph.run("match (s:`person`)-[p:`remittance`]-(o:`person`) where s.personID= '{}' return p.amount".format(value)).data()
    return len([i['p.amount'] for i in cache if int(i['p.amount'])>=4000])

def userTxConditionNum(value):
    defaultNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='OVER_DUE' and s.personID= '{}' return o.status".format(value)).data())
    clearNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='CLEAR' and s.personID= '{}' return o.status".format(value)).data())
    rejectedNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='REJECT' and s.personID= '{}' return o.status".format(value)).data())
    return defaultNum,clearNum,rejectedNum


# 生成数据

In [112]:
userID = []
userFlag = []
userStatus =[]
# 所有用户状态
userEntity = graph.run("match (s:`person`)  return s.personID,s.flag,s.status").data()
for i in userEntity:
    userID.append(i['s.personID'])
    userFlag.append(i['s.flag'])
    userStatus.append(i['s.status'])

In [113]:
n1Num = []
n2Num = []
phoneNum = []
rmtNum = []
txdNum = []
txcNum = []
txrNum = []
i = 0
for value in userID[:10000]:
    i+=1
    n1Num.append(network1FlagNum(value=value))
    n2Num.append(network2FlagNum(value=value))
    phoneNum.append(userPhoneNum(value=value))
    rmtNum.append(userRmtNum(value=value))
    c1,c2,c3 = userTxConditionNum(value=value)
    txdNum.append(c1)
    txcNum.append(c2)
    txrNum.append(c3)
    if i%50 == 0:
        print(i)

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050
9100
9150
9200
9250
9300
9350
9400
9450
9500
9550
9600
9650
9700
9750
9800
9850
9900
9950


In [117]:
import pandas as pd
df = pd.DataFrame()
df['userID'] = userID[:10000]
df['是否在黑名单'] = userFlag[:10000]
df['历史用户评分'] = userStatus[:10000]
df['一层关系黑户数'] = n1Num
df['二层关系黑户数'] = n2Num
df['持有手机数'] = phoneNum
df['大额转账数'] = rmtNum
df['违约贷款数'] = txdNum
df['完结贷款数'] = txcNum
df['被拒贷款数'] = txrNum
df

,userID,是否在黑名单,历史用户评分,一层关系黑户数,二层关系黑户数,持有手机数,大额转账数,违约贷款数,完结贷款数,被拒贷款数
0,person_1,0,AA,0,0,1,7,1,1,0
1,person_2,0,A,0,0,1,3,1,0,0
2,person_3,0,A,0,0,1,1,0,0,1
3,person_4,0,AA,0,0,1,4,0,1,0
4,person_5,0,C,0,0,1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,person_102461,0,AA,0,0,1,4,1,1,0
9996,person_102462,0,A,0,0,1,3,1,0,0
9997,person_102463,0,A,0,0,1,4,0,0,1
9998,person_102464,0,AA,0,0,1,1,0,1,0


In [118]:
# df.to_csv(r'out/graphOut.csv', encoding='utf-8', index=False)

In [124]:
df1=pd.read_csv(r'out/graphOut.csv', encoding='utf-8')
df1[:2000]

,userID,是否在黑名单,历史用户评分,一层关系黑户数,二层关系黑户数,持有手机数,大额转账数,违约贷款数,完结贷款数,被拒贷款数
0,person_1,0,AA,0,0,1,7,1,1,0
1,person_2,0,A,0,0,1,3,1,0,0
2,person_3,0,A,0,0,1,1,0,0,1
3,person_4,0,AA,0,0,1,4,0,1,0
4,person_5,0,C,0,0,1,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1995,person_194513,0,A,0,0,1,0,0,1,0
1996,person_194514,0,A,1,0,1,2,0,0,0
1997,person_194515,1,D,0,0,1,6,0,1,0
1998,person_194516,0,A,0,1,1,2,0,0,1


# func test

In [53]:
def network1FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())

value = 'person_14'
network1FlagNum(value)

1

In [51]:
def network2FlagNum(value):
    return len(graph.run("match (s:`person`)-[p]-(:`person`)-[q]-(o:`person`) where type(p) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and type(q) in ['PARENT_OF','ACKNOWLEDGE','COLLEAGUE_OF'] and o.flag='1' and s.personID= '{}' return o.personID".format(value)).data())
value = 'person_16'
network2FlagNum(value)

1

In [66]:
def userPhoneNum(value):
    return len(graph.run("match (s:`person`)-[p:`ownPhone`]-(o:`phone`) where s.personID= '{}' return o.phone".format(value)).data())
userPhoneNum(value='person_333')

1

In [81]:
value='person_14'
def userRmtNum(value):
    # 交易金额大于4000才计数
    cache = graph.run("match (s:`person`)-[p:`remittance`]-(o:`person`) where s.personID= '{}' return p.amount".format(value)).data()
    return len([i['p.amount'] for i in cache if int(i['p.amount'])>=4000])

userRmtNum(value)

4

In [94]:
value='person_1'
def userTxConditionNum(value):
    defaultNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='OVER_DUE' and s.personID= '{}' return o.status".format(value)).data())
    clearNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='CLEAR' and s.personID= '{}' return o.status".format(value)).data())
    rejectedNum = len(graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='REJECT' and s.personID= '{}' return o.status".format(value)).data())
    return defaultNum,clearNum,rejectedNum

userTxConditionNum(value)

(1, 1, 0)

In [93]:
value = 'person_15'
graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='CLEAR' and s.personID= '{}' return o.status".format(value)).data()


[{'o.status': 'CLEAR'}]

In [91]:
value = 'person_20'
graph.run("match (s:`person`)-[p:`apply`]-(o:`transaction`) where o.status='REJECT' and s.personID= '{}' return o.status".format(value)).data()


[{'o.status': 'REJECT'}]